In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain


chat = ChatOpenAI(temperature=0.1,streaming=True,callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {
        "movie": "Top Gun",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "movie": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝",
    },
    {
        "movie": "Don't look up",
        "answer":"🚫👀⬆️",
    },
]
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)


exaple_prompt = ChatPromptTemplate.from_messages([
    
    ("human", "Please reply with three emojis that represent the movie {movie}."),    
    ("ai", "{answer}")
])

prompt = FewShotChatMessagePromptTemplate(
    example_prompt=exaple_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a cinema enthusiast."),
        prompt,
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "Please reply with three emojis that represent the movie {movie}."),
    ]
)
chain = LLMChain(
    llm=chat,
    memory=memory,
    prompt=final_prompt
)

chain.invoke({"movie":"The Martian"})

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)

In [2]:
chain.invoke({"movie":"inception"})

🧠🌀🌌

{'movie': 'inception',
 'chat_history': [HumanMessage(content='The Martian'),
  AIMessage(content='🚀🌌🥔'),
  HumanMessage(content='inception'),
  AIMessage(content='🧠🌀🌌')],
 'text': '🧠🌀🌌'}

In [3]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='The Martian'),
  AIMessage(content='🚀🌌🥔'),
  HumanMessage(content='inception'),
  AIMessage(content='🧠🌀🌌')]}

In [10]:
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])
chain = LLMChain(
    llm=chat,
    memory=memory,
    prompt=prompt
)
chain.predict(question="what is the movie I asked first?")

The movie you asked about first is "The Martian."

'The movie you asked about first is "The Martian."'